# OPF Case Preparation

In [1]:
import andes
import ams

In [2]:
!ams misc --version

Python   3.12.0
ams      1.0.9
andes    1.9.3.post51+g005049af
numpy    2.0.2
cvxpy    1.6.0
solvers  CLARABEL, ECOS, ECOS_BB,
         GUROBI, HIGHS, MOSEK, OSQP,
         PIQP, SCIP, SCIPY, SCS


In [3]:
%matplotlib inline

In [4]:
andes.config_logger(stream_level=40)
ams.config_logger(stream_level=20)

In [5]:
sp = ams.load('./../cases/case_ACTIVSg200.m',
              setup=True, no_output=True,
              default_config=True)

# manually turn on all generators
sp.StaticGen.alter(src='u', idx=sp.StaticGen.get_all_idxes(),
                   value=1)

Parsing input file "./../cases/case_ACTIVSg200.m"...
CASE_ACTIVSG200  Synthetic Illinois 200-bus power system model.
Input file parsed in 0.0205 seconds.
Zero Line parameters detected, adjusted to default values: rate_b, rate_c, amax, amin.
System set up in 0.0030 seconds.


True

In [6]:
sp2 = ams.load('./../cases/ACTIVSg200.RAW',
               setup=False, no_output=True,
               default_config=True)

Parsing input file "./../cases/ACTIVSg200.RAW"...


Input file parsed in 0.0743 seconds.


In [7]:
stg_mp = set(sp.StaticGen.get_all_idxes())
stg_raw = set(sp2.StaticGen.get_all_idxes())
stg_equal = stg_mp == stg_raw
stg_equal

True

The two cases have idential generators, so we can add generator costs to the PSSE read case.

In [8]:
for index, row in sp.GCost.cache.df_in.iterrows():
    sp2.add(model='GCost', **row)

In [9]:
stg = sp2.StaticGen.get_all_idxes()
sp2.StaticGen.alter(src='gentype', attr='v', idx=stg,
                    value=sp.StaticGen.get(src='gentype', idx=stg))

sp2.StaticGen.alter(src='genfuel', attr='v', idx=stg,
                    value=sp.StaticGen.get(src='genfuel', idx=stg))

True

In [10]:
ams.io.xlsx.write(sp2,
                  './../cases/IL200_opf_20wt.xlsx',
                  overwrite=True)

xlsx file written to "./../cases/IL200_opf_20wt.xlsx"


True

The rest two OPF cases are manually prepared by changing some StaticGen's ``gentype`` and ``genfule``.